In [1]:
import os
import sys
import pandas as pd

from lib.display import Display
from lib.disk import Disk

Display.header('Challenges', 'h1')

disk = Disk('others')
table_df = disk.read_by_label('challenges-2022-01-28', '|')
table_df.columns = table_df.columns.str.replace(' ', '')
table_df = table_df.drop([0, 1]).reset_index(drop=True)

Display.frame(table_df, 5)

,account_id,value
0,b28b8e65-131d-4d5e-972f-667f5a3e6f09,"{""ecov"": [{""name"": ""activate-challenges"", ""completedAt"": ""2022-01-11T13:22:16.083Z""}, {""name"": ""vaccination"", ""completedAt"": ""2022-01-11T13:22:16.672Z""}]}"
1,007082be-12d2-4cce-a82d-9ce5eface27f,"{""ecov"": [{""name"": ""activate-challenges"", ""completedAt"": ""2022-01-14T09:46:50.794Z""}, {""name"": ""vaccination"", ""completedAt"": ""2022-01-14T09:46:51.288Z""}]}"
2,d4dff97a-16f7-4f99-9d0e-0b0e16124036,"{""ecov"": [{""name"": ""activate-challenges"", ""completedAt"": ""2022-01-12T20:46:28.136Z""}, {""name"": ""vaccination"", ""completedAt"": ""2022-01-12T20:46:29.123Z""}]}"
3,2650886b-d454-4397-b3b5-c91f735dcb47,"{""ecov"": [{""name"": ""activate-challenges"", ""completedAt"": ""2022-01-12T09:47:54.602Z""}, {""name"": ""vaccination"", ""completedAt"": ""2022-01-12T09:47:55.271Z""}]}"
4,83225437-1b2c-4f76-afff-860e39e69ffa,"{""ecov"": [{""name"": ""activate-challenges"", ""completedAt"": ""2022-01-12T09:48:20.233Z""}, {""name"": ""vaccination"", ""completedAt"": ""2022-01-12T09:48:20.604Z""}]}"


In [4]:
import json
from dateutil.parser import parse

def fix_date_value(s):
    s = s.replace('Dezember', 'December')
    return parse(s).strftime('%Y-%m-%d')

def parse_value(s):
    y = json.loads(s)
    d = {}
    for e in y['ecov']:
        key = e['name']
        value = fix_date_value(e['completedAt'])
        d[key] = value
    return d

def get_challenge_name(value, s):
    d = parse_value(value)
    return d[s] if s in d.keys() else None

# df = table_df[['account_id']]
df = table_df.copy()
df['activation'] = table_df.apply(lambda row : get_challenge_name(row['value'], 'activate-challenges'), axis = 1)
df['c1'] = table_df.apply(lambda row : get_challenge_name(row['value'], 'vaccination'), axis = 1)
df['c2'] = table_df.apply(lambda row : get_challenge_name(row['value'], 'activity'), axis = 1)
df['c3'] = table_df.apply(lambda row : get_challenge_name(row['value'], 'sharing'), axis = 1)
df['c4'] = table_df.apply(lambda row : get_challenge_name(row['value'], 'follow-up'), axis = 1)

Display.frame(df, 4)

print(f'Activated challenges: {df[~pd.isnull(df["activation"])].shape[0]}')
print(f'Completion of 1st challenge: {df[~pd.isnull(df["c1"])].shape[0]}')
print(f'Completion of 2nd challenge: {df[~pd.isnull(df["c2"])].shape[0]}')
print(f'Completion of 3rd challenge: {df[~pd.isnull(df["c3"])].shape[0]}')
print(f'Completion of 4th challenge: {df[~pd.isnull(df["c4"])].shape[0]}')

wrong_order_df = df[(
    (pd.isnull(df["c1"]) & (~pd.isnull(df["c2"]) | ~pd.isnull(df["c3"]) | (~pd.isnull(df["c4"])))) |
    (pd.isnull(df["c2"]) & (~pd.isnull(df["c3"]) | ~pd.isnull(df["c4"]))) |
    (pd.isnull(df["c3"]) & ~pd.isnull(df["c4"]))
)]

Display.frame(wrong_order_df, 4)
print(f'Problematic status for {wrong_order_df.shape[0]} users')

,account_id,value,activation,c1,c2,c3,c4
0,b28b8e65-131d-4d5e-972f-667f5a3e6f09,"{""ecov"": [{""name"": ""activate-challenges"", ""completedAt"": ""2022-01-11T13:22:16.083Z""}, {""name"": ""vaccination"", ""completedAt"": ""2022-01-11T13:22:16.672Z""}]}",2022-01-11,2022-01-11,None,None,None
1,007082be-12d2-4cce-a82d-9ce5eface27f,"{""ecov"": [{""name"": ""activate-challenges"", ""completedAt"": ""2022-01-14T09:46:50.794Z""}, {""name"": ""vaccination"", ""completedAt"": ""2022-01-14T09:46:51.288Z""}]}",2022-01-14,2022-01-14,None,None,None
2,d4dff97a-16f7-4f99-9d0e-0b0e16124036,"{""ecov"": [{""name"": ""activate-challenges"", ""completedAt"": ""2022-01-12T20:46:28.136Z""}, {""name"": ""vaccination"", ""completedAt"": ""2022-01-12T20:46:29.123Z""}]}",2022-01-12,2022-01-12,None,None,None
3,2650886b-d454-4397-b3b5-c91f735dcb47,"{""ecov"": [{""name"": ""activate-challenges"", ""completedAt"": ""2022-01-12T09:47:54.602Z""}, {""name"": ""vaccination"", ""completedAt"": ""2022-01-12T09:47:55.271Z""}]}",2022-01-12,2022-01-12,None,None,None


Activated challenges: 1000
Completion of 1st challenge: 936
Completion of 2nd challenge: 263
Completion of 3rd challenge: 119
Completion of 4th challenge: 51


,account_id,value,activation,c1,c2,c3,c4
81,c38bac12-1c21-4874-aaee-594768d9755e,"{""ecov"": [{""name"": ""sharing"", ""completedAt"": ""9. November 2021""}, {""name"": ""activate-challenges"", ""completedAt"": ""9. November 2021""}, {""name"": ""vaccination"", ""completedAt"": ""9. November 2021""}, {""name"": ""follow-up"", ""completedAt"": ""10. November 2021""}]}",2021-11-09,2021-11-09,None,2021-11-09,2021-11-10
123,f0190f16-e748-424e-9a3d-7b7d52f69ab9,"{""ecov"": [{""name"": ""sharing"", ""completedAt"": ""11. November 2021""}]}",None,None,None,2021-11-11,None
131,3f6a8197-29f4-4caf-95b2-b15b2acead9a,"{""ecov"": [{""name"": ""sharing"", ""completedAt"": ""12. November 2021""}, {""name"": ""activate-challenges"", ""completedAt"": ""12. November 2021""}, {""name"": ""vaccination"", ""completedAt"": ""12. November 2021""}]}",2021-11-12,2021-11-12,None,2021-11-12,None
137,3190362b-d4ac-40a3-9ef7-1c98d72829fa,"{""ecov"": [{""name"": ""sharing"", ""completedAt"": ""13. November 2021""}, {""name"": ""activate-challenges"", ""completedAt"": ""13. November 2021""}, {""name"": ""vaccination"", ""completedAt"": ""13. November 2021""}]}",2021-11-13,2021-11-13,None,2021-11-13,None


Problematic status for 70 users


In [5]:
wo_df1 = df[(pd.isnull(df["c1"]) & (~pd.isnull(df["c2"]) | ~pd.isnull(df["c3"]) | (~pd.isnull(df["c4"]))))]
print(f'Completed 2nd/3rd/4th without the 1st challenge: {wo_df1.shape[0]}')
wo_df2 = df[(pd.isnull(df["c2"]) & (~pd.isnull(df["c3"]) | ~pd.isnull(df["c4"])))]
print(f'Completed 3rd/4th without the 2nd challenge: {wo_df2.shape[0]}')
wo_df3 = df[(pd.isnull(df["c3"]) & ~pd.isnull(df["c4"]))]
print(f'Completed 4th without the 3rd challenge: {wo_df3.shape[0]}')

Completed 2nd/3rd/4th without the 1st challenge: 25
Completed 3rd/4th without the 2nd challenge: 70
Completed 4th without the 3rd challenge: 0


In [6]:
df[(pd.isnull(df["c1"]) & (~pd.isnull(df["c3"])))].shape[0]

25

In [7]:
df[(pd.isnull(df["c2"]) & (~pd.isnull(df["c3"])))].shape[0]

70